### Notebook: functions to take raw .csvs downloaded from sheet and put them into a readable format. Output files are in `Processed CSVs` folder

In [4]:
# Imports
import pandas as pd
import numpy as np


In [2]:
def make_blocks(data):
    '''
    Function that takes raw df with "Trace name" as in position [0,0] and returns the blocks
    '''
    # Each block has 5 columns: Trace name, ID, On time, Freq, Tags
    num_cols = data.shape[1]
    num_rows = data.shape[0]    
    block_size = 6
    # Separate into blocks
    blocks = []
    for start in range(0, num_cols, block_size):  
        end = start + block_size
        if (end <= num_cols) & (data.iloc[0,start] == "Trace name"):
            block = data.iloc[:, start:end]
            # print(block)
            block.columns = ['Trace name', 'ID', 'On time', 'Freq', 'Tags','Type']
            if start == 0:
                block['Type']=['Inhibitory']*num_rows
            elif start == 6:
                block['Type']=['Excitatory']*num_rows
            elif start == 12:
                block['Type']=['Cell-attached']*num_rows
            blocks.append(block)
    # Combine all blocks into one DataFrame
    df_clean = pd.concat(blocks, ignore_index=True)

    # Drop rows that have NaN in Trace name or ID
    df_clean.dropna(subset=['Trace name', 'ID'], how='all', inplace=True)

    df_clean = df_clean.dropna(how='all')
    return df_clean


def prepare_df(df):
    '''
    Function to convet column data types to feed make_ waveworms function
    '''
    df['Trace name'] = df['Trace name'].astype('string')
    df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
    df["Freq"] = pd.to_numeric(df["Freq"], errors="coerce")
    df["Seconds"] = df['On time']*0.001
    return df


def trace_filter(df):
    pattern = r'^\d{4}_\d{2}_\d{2}_\d{4}$'
    index = df['Trace name'].str.match(pattern, na=False)
    df_filtered = df[index]
    return df_filtered

def create_df(file_path):
    '''
    Complete function that takes csv file path from excel spreadsheets and turns it into a readable dataframe that show event times

    '''

    df = pd.read_csv(file_path,header=None)

    # Read the raw CSV , headers are irregular
    data = df.iloc[1:,1:]
    # Run all of the functions to process files
    df_clean = make_blocks(data)
    prep_df = prepare_df(df_clean)
    df_filtered = trace_filter(prep_df)
    df_filtered = df_filtered.dropna(how='all')

    return df_filtered

In [7]:
sheet_names_df = pd.read_csv('/Users/haleyoro/Desktop/murray-neuroscience-lab/Processed CSVs/sheet_names.csv', header=None)
sheet_names = sheet_names_df.iloc[:,0].values
sheet_names

array(['2012_04_25_cell1', '2012_04_25_cell3', '2012_04_27_cell1',
       '2012_06_22_cell3', '2012_06_25_cell3', '2012_06_29',
       '2012_08_01_cell1', '2012_08_01_cell3', '2012_08_07',
       '2012_08_31_cell1', '2012_08_31_cell2', '2012_08_31_cell3',
       '2012_08_31_cell4', '2012_10_04_cell2', '2012_12_03_cell1',
       '2012_12_04_cell1', '2012_12_04_cell2', '2012_12_05_cell1',
       '2012_12_05_cell3', '2012_12_05_cell4', '2012_12_06_cell1',
       '2012_12_06_cell2', '2012_12_06_cell3', '2012_12_06_cell4',
       '2012_12_06_cell5', '2012_12_06_cell6', '2013_03_20_cell1',
       '2013_03_21_cell1', '2013_03_21_cell2', '2013_03_21_cell3',
       '2013_03_21_cell4', '2013_03_21_cell5', '2013_03_22_cell1',
       '2013_03_22_cell2', '2013_03_22_cell3', '2013_03_22_cell4',
       '2013_03_22_cell5', '2013_03_22_cell6', '2012_10_04_cell1'],
      dtype=object)

Load names of all .abf files in the shared Google folder

In [11]:
abfs = pd.read_csv('/Users/haleyoro/Desktop/murray-neuroscience-lab/Excel processor/all_abf_files.csv',header=None)
abfs = abfs[0].to_numpy()
abfs


array(['2013_03_22_0045.abf', '2013_03_22_0042.abf',
       '2013_03_22_0039.abf', ..., '2012_08_29_0009.abf',
       '2012_08_29_0016.abf', '2012_08_29_0018.abf'],
      shape=(1050,), dtype=object)

In [15]:
# df_filtered = create_df('/Users/haleyoro/Downloads/Exported_CSVs/2012_10_04_cell2.csv')
# df_filtered.to_csv("2012_10_04_cell2.csv", index=False)

In [38]:
df_filtered

,Trace name,ID,On time,Freq,Tags,Type,Seconds
1,2012_10_04_0029,1,12799.27,NaN,bout start; shock,Inhibitory,12.79927
2,2012_10_04_0029,2,12818.74,51.3611,NaN,Inhibitory,12.81874
3,2012_10_04_0029,3,12835.24,60.6061,NaN,Inhibitory,12.83524
4,2012_10_04_0029,4,12854.46,52.0291,NaN,Inhibitory,12.85446
5,2012_10_04_0029,5,12874.93,48.8520,NaN,Inhibitory,12.87493
...,...,...,...,...,...,...,...
790,2012_10_04_0024,102,29028.78,27.7546,bout end,Cell-attached,29.02878
791,2012_10_04_0024,103,35641.77,0.1512,bout start; spontaneous,Cell-attached,35.64177
792,2012_10_04_0024,104,35662.12,49.1400,NaN,Cell-attached,35.66212
793,2012_10_04_0024,105,35684.59,44.5038,NaN,Cell-attached,35.68459


One function to run all the functions together

In [39]:
for name in sheet_names:
    file_path = "/Users/haleyoro/Downloads/Exported_CSVs/"+name + '.csv'
    df_filtered = create_df(file_path)
    df_filtered.to_csv(name+".csv", index=False)